In [1]:
# import requests
# import json

# lat, lon = 35.9786, 126.7114

# url = (
#     f"https://climate-api.open-meteo.com/v1/climate?"
#     f"latitude={lat}&longitude={lon}"
#     f"&start_year=2020&end_year=2020"
#     f"&models=CMIP6"
#     f"&temperature_2m_mean=true"
#     f"&relative_humidity_2m_mean=true"
#     f"&cloudcover_mean=true"
#     f"&windspeed_10m_mean=true"
# )

# response = requests.get(url)

# if response.status_code == 200:
#     data = response.json()
#     print("✅ API 호출 성공!")
#     print(json.dumps(data, indent=2, ensure_ascii=False))
# else:
#     print(f"❌ API 호출 실패: {response.status_code}")
#     print(response.text)

In [2]:
# import requests

# API_KEY = "13a24ecc3a2785b9652b8816b02a8e55"
# lat = 35.9786
# lon = 126.7114
# start = 1609459200   # 2021-01-01 00:00:00 UTC
# end = 1609545600     # 2021-01-02 00:00:00 UTC

# url = f"https://history.openweathermap.org/data/2.5/history/city?lat={lat}&lon={lon}&type=hour&start={start}&end={end}&appid={API_KEY}"

# response = requests.get(url)

# if response.status_code == 200:
#     print("✅ 역사적 날씨 데이터 수신 성공!")
#     print(response.json())
# else:
#     print(f"❌ 실패 - 코드 {response.status_code}")
#     print(response.text)

In [3]:
pip install meteostat

Note: you may need to restart the kernel to use updated packages.


In [4]:
# from meteostat import Point, Daily
# from datetime import datetime

# # 위치: 군산
# location = Point(35.9786, 126.7114)

# # 조회 기간 (2023년 3월 1일 ~ 31일)
# start = datetime(2023, 3, 1)
# end = datetime(2023, 3, 31)

# data = Daily(location, start, end)
# data = data.fetch()

# print(data.head())

In [5]:
# from meteostat import Point, Daily
# from datetime import datetime
# import pandas as pd

# # 위치: 군산
# location = Point(35.9786, 126.7114)

# # 기간
# start = datetime(2023, 3, 1)
# end = datetime(2023, 3, 31)

# # 데이터 가져오기
# data = Daily(location, start, end).fetch()

# # ✨ 날짜 컬럼이 문자열일 경우 대비: 인덱스를 datetime으로 변환
# data.index = pd.to_datetime(data.index)

# # 확인
# print(data.head())
# # print(f"\n✅ 인덱스 타입 확인: {type(data.index)}")

In [6]:
# response = requests.get("https://ipapi.co/json/").json()
# print(json.dumps(response, indent=2, ensure_ascii=False))

In [7]:
# import requests
# def get_lat_lon_from_ip():
#     """IP 기반으로 위도와 경도, 도시명 반환"""
#     try:
#         response = requests.get("https://ipinfo.io/json", timeout=5)
#         data = response.json()
#         lat, lon = map(float, data["loc"].split(","))
#         city = data.get("city", "알 수 없음")
#         print(f"📍 자동 위치 감지: {city} (위도 {lat}, 경도 {lon})")
#         return lat, lon, city
#     except Exception as e:
#         print("❌ IP 기반 위치 조회 실패:", e)
#         return None, None, None
# # 👇 함수 호출 (출력 테스트)
# get_lat_lon_from_ip()

In [8]:
from meteostat import Point, Daily
from datetime import datetime, timedelta
import pandas as pd
import requests

# 1. IP 기반으로 위도, 경도, 도시명 가져오기
def get_lat_lon_from_ip():
    """IP 기반으로 위도와 경도, 도시명 반환"""
    try:
        response = requests.get("https://ipinfo.io/json", timeout=5)
        data = response.json()
        lat, lon = map(float, data["loc"].split(","))
        city = data.get("city", "알 수 없음")
        print(f"📍 자동 위치 감지: {city} (위도 {lat}, 경도 {lon})")
        return city, lat, lon
    except Exception as e:
        print("❌ IP 기반 위치 조회 실패:", e)
        return None, None, None

# 2. 도시명을 위도/경도로 변환 (수동 입력 시)
def get_coords_from_city(city_name):
    try:
        url = f"https://nominatim.openstreetmap.org/search?q={city_name}&format=json&limit=1"
        headers = {"User-Agent": "weather-capstone-app"}
        response = requests.get(url, headers=headers).json()

        if response:
            lat = float(response[0]["lat"])
            lon = float(response[0]["lon"])
            return city_name, lat, lon
        else:
            raise ValueError("도시 이름을 찾을 수 없습니다.")
    except Exception as e:
        print("❌ 도시명 변환 실패:", e)
        new_city = input("🚨 유효한 도시명을 다시 입력해 주세요: ")
        return get_coords_from_city(new_city)

# 3. 사용자 위치 자동 감지 또는 수동 입력
def get_location():
    city, lat, lon = get_lat_lon_from_ip()
    if city is None or lat is None or lon is None:
        user_city = input("🔎 자동 위치 조회에 실패했습니다. 도시명을 직접 입력해 주세요 (예: Seoul): ")
        city, lat, lon = get_coords_from_city(user_city)
    return city, lat, lon

today = datetime.today()
start = today - timedelta(days=30)
end = today

# 5. 위치 조회 및 출력
city, lat, lon = get_location()
print(f"📍 사용된 위치: {city} (위도 {lat}, 경도 {lon})")
print(f"📅 조회 기간: {start.date()} ~ {end.date()}")

# 6. Meteostat 날씨 데이터 수집
location = Point(lat, lon)
data = Daily(location, start, end).fetch()
data.index = pd.to_datetime(data.index)

# 7. 평균값 계산 및 출력
mean_values = data.mean(numeric_only=True)
print("\n📊 31일 평균 기상 데이터:")
print(mean_values.round(2))

📍 자동 위치 감지: Gunsan (위도 35.9786, 경도 126.7114)
📍 사용된 위치: Gunsan (위도 35.9786, 경도 126.7114)
📅 조회 기간: 2025-03-21 ~ 2025-04-20

📊 31일 평균 기상 데이터:
tavg      10.17
tmin       5.17
tmax      15.68
prcp       2.05
snow        NaN
wdir     231.27
wspd      16.70
wpgt        NaN
pres    1012.86
tsun        NaN
dtype: float64


In [9]:
# 1. FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated...
# 날짜 형식의 자동 파싱 기능이 곧 제거될 예정
# pandas 측에서 사전 경고

# 2. Warning: Cannot load daily/RKSK0.csv.gz from https://bulk.meteostat.net/v2/
# meteostat의 특정 관측소에서 압축된 일일 데이터가 부분적인 손상이나 다운로드 불가능한 오류

In [10]:
import json

# 평균 기상 데이터 (pandas Series → dict)
avg_weather_dict = mean_values.round(2).to_dict()

# 저장할 파일 경로
filename = f"monthly_avg_weather_{city}_{today.strftime('%Y%m%d')}.json"

# JSON으로 저장
with open(filename, "w", encoding="utf-8") as f:
    json.dump(avg_weather_dict, f, ensure_ascii=False, indent=2)

print(f"✅ 월평균 기상 데이터를 파일로 저장했습니다: {filename}")

✅ 월평균 기상 데이터를 파일로 저장했습니다: monthly_avg_weather_Gunsan_20250420.json
